In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy import stats

In [4]:
data=pd.read_excel("Gold.xlsx")

In [5]:
df=pd.DataFrame(data)

In [6]:
df.head()

,Year,Gold Price (10g),Rate of Change (%),Imports value,Exports Value,Imports Percentage (%),Exports Percentage (%),Growth Rate (Gold Exports),GDP,GOLD RESERVES (IN CRORE),Exchange Rate(USD-INR),CPI Inflation Rate (%),Repo Rate (%),Unemployment Rate (%)
0,1999,4234.0,1.92,3180500,636100,0.90,0.0017,-,6.2,12559,43.060000,4.67,8.00,5.4
1,2000,4400.0,3.92,3300000,660000,0.91,0.0022,0.0375,4.0,12973,44.948154,4.01,9.00,5.6
2,2001,4300.0,-2.27,3450000,690000,0.92,0.0026,0.0455,5.2,12711,47.172949,3.78,8.75,5.8
3,2002,4990.0,16.05,3600000,720000,0.92,0.0025,0.0435,5.4,14868,48.575595,4.30,5.75,5.9
4,2003,5600.0,12.22,4200000,840000,0.92,0.0025,0.1667,4.0,16785,46.538938,3.81,5.00,5.6


In [7]:
# Assuming the data contains two sheets: 'Gold Prices' and 'GDP Data'
gold_data = df['Gold Price (10g)']
gdp_data = df['GDP']


In [8]:
# Display first few rows of both datasets for inspection
print(gold_data.head())
print(gdp_data.head())


0    4234.0
1    4400.0
2    4300.0
3    4990.0
4    5600.0
Name: Gold Price (10g), dtype: float64
0    6.2
1    4.0
2    5.2
3    5.4
4    4.0
Name: GDP, dtype: float64


In [9]:
# 1. Data Cleaning
## Handling Missing Values
# For gold price and GDP data, fill missing values using forward-fill or interpolation
gold_data = gold_data.fillna(method='ffill')
gdp_data = gdp_data.fillna(method='ffill')


In [10]:
# Drop rows with missing values if any (e.g., if 'ffill' does not work perfectly)
gold_data = gold_data.dropna()
gdp_data = gdp_data.dropna()

In [11]:
## Removing Duplicates
gold_data = gold_data.drop_duplicates()
gdp_data = gdp_data.drop_duplicates()

In [12]:
# 2. Data Transformation

# Convert date column to datetime (if it exists)
if 'date' in data.columns:
    data['date'] = pd.to_datetime(data['date'])
    data.set_index('date', inplace=True)

In [13]:
# Feature Engineering: Create gold price percentage change
data['gold_price_pct_change'] = data['Gold Price (10g)'].pct_change() * 100  # Percentage change
data['GDP_growth'] = data['GDP'].pct_change() * 100  # GDP growth

In [14]:
# Dropping rows with NaN values generated by pct_change (first row usually)
data.dropna(inplace=True)

In [15]:
# Normalize or scale numerical features
scaler = StandardScaler()  # You can also use MinMaxScaler() if preferred
outliers=data[['Gold Price (10g)', 'GDP']] = scaler.fit_transform(data[['Gold Price (10g)', 'GDP']])

In [16]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = data['Gold Price (10g)'].quantile(0.25)
Q3 = data['GDP'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = (data['Gold Price (10g)'] < lower_bound) | (data['Gold Price (10g)'] > upper_bound)
outliers = (data['GDP'] < lower_bound) | (data['GDP'] > upper_bound)

# Remove outliers
data = data[~outliers]

print(data)

    Year  Gold Price (10g)  Rate of Change (%)  Imports value  Exports Value  \
1   2000         -1.193982                3.92        3300000         660000   
2   2001         -1.199432               -2.27        3450000         690000   
3   2002         -1.161828               16.05        3600000         720000   
4   2003         -1.128583               12.22        4200000         840000   
5   2004         -1.114958                4.46        4800000         960000   
6   2005         -1.052285               19.66        5250000        1050000   
7   2007         -0.845188               54.29        6300000        1260000   
8   2008         -0.752540               15.74        8100000        1620000   
9   2009         -0.643542               16.00        9375000        1875000   
10  2010         -0.425546               27.59       10875000        2175000   
11  2011          0.004996               42.70       13875000        2775000   
12  2012          0.258416              

In [17]:
# 3. Feature Engineering
# Example: Calculate percentage change in gold prices (assuming 'Gold_Price' column exists)
df['Gold_Price_pct_change'] = df['Gold Price (10g)'].pct_change() * 100

In [18]:
# Example: Calculate GDP growth rate (assuming 'GDP' column exists)
df['GDP_growth'] = df['GDP'].pct_change() * 100

In [19]:
# 4. Outlier Detection & Handling (using Z-score)
from scipy.stats import zscore


In [20]:
# 4. Outlier Detection & Handling (using Z-score)
# Calculate Z-scores for each numerical column individually
numerical_columns = df.select_dtypes(include=[np.number]).columns


In [21]:
# Iterate over the numerical columns to calculate Z-scores
for col in numerical_columns:
    df[f'{col}_zscore'] = np.abs(zscore(df[col]))


In [22]:
# Remove rows with Z-scores greater than a threshold (e.g., 3)
df = df[(df[[f'{col}_zscore' for col in numerical_columns]] < 3).all(axis=1)]

In [23]:
# Drop the Z-score columns after handling outliers
df.drop(columns=[f'{col}_zscore' for col in numerical_columns], inplace=True)

In [24]:
# 5. Data Normalization (using Min-Max Scaling)
# Ensure the relevant columns exist
scaled_columns = ['Gold Price (10g)', 'GDP', 'Gold_Price_pct_change', 'GDP_growth']  # Adjust as necessary

In [25]:
# Check if columns exist
missing_columns = [col for col in scaled_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"The following columns are missing in the dataset: {missing_columns}")

In [26]:
# Check the shape of the data before scaling
print(f"Data shape before scaling: {df[scaled_columns].shape}")


Data shape before scaling: (0, 4)


In [27]:
# 6. Save cleaned data
df.to_csv('cleaned_gold_gdp_data.csv', index=False)

# Display the cleaned data
print(df.head())

Empty DataFrame
Columns: [Year, Gold Price (10g), Rate of Change (%), Imports value, Exports Value, Imports Percentage (%), Exports Percentage (%), Growth Rate (Gold Exports), GDP, GOLD RESERVES (IN CRORE), Exchange Rate(USD-INR) , CPI Inflation Rate (%), Repo Rate (%), Unemployment Rate (%), Gold_Price_pct_change, GDP_growth]
Index: []
